In [1]:
import pandas as pd

In [2]:
ava_df = pd.read_csv('AVA.txt', sep=' ', names=['Image ID', 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 'Semantic ID 1', 'Semantic ID 2', 'Challenge ID'])
ava_df.head()

,Image ID,1,2,3,4,5,6,7,8,9,10,Semantic ID 1,Semantic ID 2,Challenge ID
1,953619,0,1,5,17,38,36,15,6,5,1,1,22,1396
2,953958,10,7,15,26,26,21,10,8,1,2,1,21,1396
3,954184,0,0,4,8,41,56,10,3,4,0,0,0,1396
4,954113,0,1,4,6,48,37,23,5,2,2,15,21,1396
5,953980,0,3,6,15,57,39,6,1,1,1,22,38,1396


In [3]:
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm

# Filter Bad Images

Takes 30 minutes

In [4]:
%%time
import numpy as np
import threading

# Catch all warnings too
import warnings
warnings.filterwarnings("error")

N = len(ava_df)    

rows_to_delete = []

def filter_bad_rows(df):
    for i, row in df.iterrows():
        try:
            im = Image.open('images/{}.jpg'.format(row['Image ID'])).convert('RGB')
            im.close()
        except:
            rows_to_delete.append(i) 

t1 = threading.Thread(target=filter_bad_rows, args=(ava_df.iloc[range(N//4)],))
t2 = threading.Thread(target=filter_bad_rows, args=(ava_df.iloc[range(N//4, N//2)],))
t3 = threading.Thread(target=filter_bad_rows, args=(ava_df.iloc[range(N//2, 3*N//4)],))
t4 = threading.Thread(target=filter_bad_rows, args=(ava_df.iloc[range(3*N//4, N)],))

t1.start()
t2.start()
t3.start()
t4.start()

t1.join()
t2.join()
t3.join()
t4.join()

print("{} Bad rows".format(len(rows_to_delete)))
# for i, row in tqdm(ava_df.iterrows()):
#     try:
#         im = Image.open('images/{}.jpg'.format(row['Image ID']))
#         im.verify()
#         im.close()
#     except:
#         print("Row {} is bad".format(i))
#         rows_to_delete.append(i)   

Exception ignored in: <_io.FileIO name='images/501095.jpg' mode='rb' closefd=True>
Exception ignored in: <_io.FileIO name='images/864006.jpg' mode='rb' closefd=True>
Exception ignored in: <_io.FileIO name='images/706063.jpg' mode='rb' closefd=True>
Exception ignored in: <_io.FileIO name='images/547917.jpg' mode='rb' closefd=True>
Exception ignored in: <_io.FileIO name='images/570175.jpg' mode='rb' closefd=True>
Exception ignored in: <_io.FileIO name='images/412749.jpg' mode='rb' closefd=True>
Exception ignored in: <_io.FileIO name='images/887750.jpg' mode='rb' closefd=True>
Exception ignored in: <_io.FileIO name='images/893908.jpg' mode='rb' closefd=True>
Exception ignored in: <_io.FileIO name='images/937294.jpg' mode='rb' closefd=True>
Exception ignored in: <_io.FileIO name='images/899625.jpg' mode='rb' closefd=True>
Exception ignored in: <_io.FileIO name='images/759039.jpg' mode='rb' closefd=True>
Exception ignored in: <_io.FileIO name='images/912486.jpg' mode='rb' closefd=True>
Exce

70 Bad rows
CPU times: user 35min 59s, sys: 58.3 s, total: 36min 58s
Wall time: 32min 52s


In [5]:
proc_ava_df = ava_df.drop(rows_to_delete)
print('Removed {} rows'.format(len(rows_to_delete)))

Removed 70 rows


In [6]:
proc_ava_df.to_csv('proc_AVA.csv', index=False)

# Calculate Mean Score

In [7]:
ava_df = pd.read_csv('proc_AVA.csv')

In [8]:
ava_df.head()

,Image ID,1,2,3,4,5,6,7,8,9,10,Semantic ID 1,Semantic ID 2,Challenge ID
0,953417,0,0,0,5,32,50,23,10,3,1,22,0,1396
1,953777,0,3,2,3,13,40,35,21,8,3,20,53,1396
2,953756,0,2,3,9,35,50,20,5,2,2,0,0,1396
3,954195,0,1,7,26,56,23,6,1,0,2,0,0,1396
4,953903,0,1,4,5,33,50,17,9,3,2,21,28,1396


In [9]:
import numpy as np

def compute_mean_score(row):
    prob_distr = row[[1,2,3,4,5,6,7,8,9,10]].values
    prob_distr = prob_distr / prob_distr.sum()
    weights = np.arange(10) + 1
    mean_score = (prob_distr * weights).sum()
    return mean_score

In [10]:
%%time
ava_df['Mean Score'] = ava_df.apply(compute_mean_score, axis=1)

CPU times: user 1min 10s, sys: 1.16 s, total: 1min 11s
Wall time: 1min 11s


In [11]:
ava_df.head()

,Image ID,1,2,3,4,5,6,7,8,9,10,Semantic ID 1,Semantic ID 2,Challenge ID,Mean Score
0,953417,0,0,0,5,32,50,23,10,3,1,22,0,1396,6.112903
1,953777,0,3,2,3,13,40,35,21,8,3,20,53,1396,6.593750
2,953756,0,2,3,9,35,50,20,5,2,2,0,0,1396,5.796875
3,954195,0,1,7,26,56,23,6,1,0,2,0,0,1396,5.040984
4,953903,0,1,4,5,33,50,17,9,3,2,21,28,1396,5.943548


In [12]:
ava_df.to_csv('proc_AVA.csv', index=False)

In [13]:
len(ava_df)

255460